In [43]:
import pandas as pd
import numpy as np

In [44]:
df=pd.read_csv('/content/diabetes.csv')

In [45]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [47]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [48]:
# finding the correlation between the output and the other columns
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [49]:
# Spliting the data
X =df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [50]:
#Scaling the dataset
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()


In [51]:
X= scaler.fit_transform(X)

In [52]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [53]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=.2,random_state=1)

In [54]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [55]:
pip install -U keras-tuner

In [56]:
import kerastuner as kt

In [57]:
# for hyperparameter tuning we have to build function
# using this for optimizer selection
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8)),
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','adadelta','rmsprop'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [58]:
tuner=kt.RandomSearch(
                build_model,
                objective='val_accuracy',
                max_trials=5,

)

# this is the creation of tuner object

In [59]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [60]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [61]:
# now for no. of neurons
def build_model(hp):

  model=Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8)),
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [62]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='diabetes'
)

In [63]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [64]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [66]:
model = tuner.get_best_models(num_models=1)[0]

In [67]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 8ms/step - loss: 0.4854 - accuracy: 0.7622 - val_loss: 0.4757 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4754 - accuracy: 0.7687 - val_loss: 0.4698 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4683 - accuracy: 0.7704 - val_loss: 0.4653 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4632 - accuracy: 0.7720 - val_loss: 0.4635 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4590 - accuracy: 0.7752 - val_loss: 0.4627 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4562 - accuracy: 0.7736 - val_loss: 0.4620 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4536 - accuracy: 0.7736 - val_loss: 0.4624 - val_accuracy: 0.8052
Epoch 14/

In [68]:
# hypertuning for how to select no of layers

def build_model(hp):

  model=Sequential()

  model.add(Dense(72, activation='relu',input_dim=8)) # Input layer
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)): # this means we sill build 10 models which have 1 layer , 2 layer and so on up to 10 layer respectively.

     model.add(Dense(72,activation='relu'))  # hidden layer

  model.add(Dense(1,activation='sigmoid'))   # output layer
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [69]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='project',
    project_name='diabetes'
)

In [70]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [71]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [72]:
model=tuner.get_best_models(num_models=1)[0]

In [73]:
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 2ms/step - loss: 0.4819 - accuracy: 0.7785
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4710 - accuracy: 0.7899
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.7899
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4502 - accuracy: 0.7964
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4323 - accuracy: 0.8046
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.7866
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.7948
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4131 - accuracy: 0.8208
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4125 - accuracy: 0.8143
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3907 - accuracy: 0.8290
Epo

In [74]:
# all in one model
def build_model(hp):
  model=Sequential()

  counter=0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
          activation=hp.Choice('activation'+str(i),values=['tanh','relu','sigmoid']),
          input_dim=8
      ))
    else:
        model.add(Dense(
          hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
          activation=hp.Choice('activation'+str(i),values=['tanh','relu','sigmoid'])
      ))
  counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [75]:
tuner=kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='main_dir',
    project_name='diabetes project'
)

In [76]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 09s


In [77]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 56,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'tanh',
 'units2': 8,
 'activation2': 'tanh',
 'units3': 8,
 'activation3': 'tanh',
 'units4': 8,
 'activation4': 'tanh',
 'units5': 8,
 'activation5': 'tanh',
 'units6': 8,
 'activation6': 'tanh'}

In [78]:
model = tuner.get_best_models(num_models=1)[0]

In [81]:
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.3562 - accuracy: 0.8388 - val_loss: 0.5148 - val_accuracy: 0.7792
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.3562 - accuracy: 0.8388 - val_loss: 0.5114 - val_accuracy: 0.7922
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 0.3565 - accuracy: 0.8453 - val_loss: 0.5173 - val_accuracy: 0.7662
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 0.3590 - accuracy: 0.8241 - val_loss: 0.5151 - val_accuracy: 0.7857
Epoch 11/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3515 - accuracy: 0.8420 - val_loss: 0.5089 - val_accuracy: 0.7727
Epoch 12/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3530 - accuracy: 0.8420 - val_loss: 0.5406 - val_accuracy: 0.7662
Epoch 13/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3525 - accuracy: 0.8322 - val_loss: 0.5333 - val_accuracy: 0.7532
Epoch 14/